# **Predictive Model for Motor Vehicle Accidents: Injury and Fatality Prevention and Intersection Safety  (January 2015 - December 2024)**

#### *By Alejandro J. Ordonez*

**Version 1.0.1**

## **Project Overview**:

This project outlines the motor vehicle (cars, trucks, buses, motorcycle) accident data in NYC, based on intersection characteristics, finding patterns within drivers, accident concentration by zip code, and commonalities among high risk areas and vehicles. 

##### **Our Goal**: 

This project aims to create a model to predict potential injury or fatalities based on variables given by NYPD public data. It also aims to create an interactive heat map for intersection collisions and provide a comprehensive look at where collisions are occuring in NYC.

Further exploration of this model could aid insurance companies in adjusting premiums for high risk profiles and locations, provide government agencies with actionable data to improve traffic safety, and give new yorkers a compreshensive look at where accidents are occuring. 

We plan on doing so by analyzing historical accident data from the last 10 years, traffic patterns, and intersection structures. The predictive model will identify high risk areas and attempt to predict future accidents within a certain margin of error. 

##### **Methodology**: 

We plan on using NYC public data in order to create this model.

We will be using SQL, and more specifically Google's RDBMS BigQuery to retrieve and clean data. 

We will be using R to help with statistical analysis on the cleaned dataset, as well as to create the logistical regression model for injury/fatality analysis. 

And finally we will be using Tableau to create a heat map of high risk areas and intersections with high counts of collisions. 

# **Data Collection:**

To get our dataset we looked on the NYC public data in order to find the motor vehicle accident data. 

We used this table for the personal profiles: https://data.cityofnewyork.us/Public-Safety/Motor-Vehicle-Collisions-Person/f55k-p6yu/about_data

We used this table for the collision in the intersection: https://data.cityofnewyork.us/Public-Safety/Motor-Vehicle-Collisions-Crashes/h9gi-nx95/about_data

We used this table for the specific vehicles in the crashes: https://data.cityofnewyork.us/Public-Safety/Motor-Vehicle-Collisions-Vehicles/bm4k-52h4/about_data

All of the data that we have collected will be stored into a personal Google Cloud and we will use Google's BigQuery in order to query the data. 

**For this project we will refer to the dataset as `insurance` and the tables as `insurance.persons`, `insurance.crashes`, and `insurance.vehicles` respectively.**

# **Data Cleaning**

As of now, SQL queries in this notebook do not run, however the final product is in the `motor-vehicle-accident-model` repo under the csv: `clean_motor_vehicle_table`. The following steps document the data cleaning that created the CSV we will be using. 

Using BigQuery, we will first clean the first table, `insurance.persons`. We will filter out null values for relevant fields and select data in between 2015-2024. 

Now we will move onto the next table `insurance.crashes` where we will do largely the same thing as we did for the previous table. 

For the last table (`insurance.vehicles`) we need to condense down the types of vehicles from over 1000 into 11 types:

(Sedan, SUV, Taxi, Pickup Truck, Van, Emergency Vehicle, Livery, Bus, Motorcycle, Bicycle, Truck, and Other)

Like the other queries, we will also filter out null values for relevant fields and select data in between 2015-2024

In this case, we used a common table expression "cleaned_vehicles" to first create a table then run the query. We will use this same methodology to combine the rest of the cleaned queries into one query and use **INNER JOIN** to combine the tables together. 

### Choosing Variables

Now that we have the cleaned tables, we need to select what we will be pulling from the new joined table. Of the variables for our model, we will pull a few that are relevant and can be factored in R. We ended up with the following fields:

* Crash date
* Crash time
* Borough
* Zip code
* Number of fatalities caused by crash
* Number of persons injured by crash
* Driver injury
* Drivers license Status
* Vehicle type
* Age
* Sex

## **Final Model Query**

We now have a cleaned dataset with all of the variables for the model. Although the code above does not work in Jupyter notebook, the result is in the `motor-vehicle-accident-model` repo in my GitHub under the csv file `clean_motor_vehicle_table`. Now we can begin with the analysis in R. 

# **Model Data Analysis**

Now we will use R to analyze the motor vehicle data we had just cleaned. First we will retrieve tidyverse and read the CSV file as "vehicle_data". 

In [ ]:
#Optional, install all packages we will be using for this analysis:
install.packages("tidyverse")
install.packages("RCurl")
install.packages("forcats")
install.packages("broom")
install.packages("ResourceSelection")

In [ ]:
library(tidyverse)
library(RCurl)
raw_file <- getURL("https://raw.githubusercontent.com/ajordonez/motor-vehicle-accident-model/refs/heads/main/cleaned_data/clean_motor_vehicle_table.csv")
vehicle_data <- read.csv(text = raw_file)
head(vehicle_data)

We will now convert the following fields into factors for easier analysis

In [ ]:
vehicle_data$BOROUGH <- as.factor(vehicle_data$BOROUGH)
vehicle_data$BOROUGH <- relevel(factor(vehicle_data$BOROUGH), ref = "STATEN ISLAND")
vehicle_data$PERSON_INJURY <- as.factor(vehicle_data$PERSON_INJURY)
vehicle_data$PERSON_SEX <- as.factor(vehicle_data$PERSON_SEX)
vehicle_data$DRIVER_LICENSE_STATUS <- factor(vehicle_data$DRIVER_LICENSE_STATUS, 
                                           levels = c("Unlicensed", "Permit", "Licensed"), 
                                           ordered = TRUE)
vehicle_data$TYPE_OF_VEHICLE <- as.factor(vehicle_data$TYPE_OF_VEHICLE)

#Make sure that for drivers license status it is ordered
levels(vehicle_data$DRIVER_LICENSE_STATUS)


We now have most of the variables as factors, but we also have age as a factor for our model. We will now check to make sure for age, no values fall out of reasonable driver age.

In [ ]:
max(vehicle_data$PERSON_AGE); min(vehicle_data$PERSON_AGE)

We can see that the minimum is 1 and maximum is 145. This is not realistic and was probably due to data entry error. We will make the minimum value 12 and maximum 100 for this analysis.


In [28]:
vehicle_data <- vehicle_data %>% filter(PERSON_AGE >= 16, PERSON_AGE < 100)

For our injury/fatality analysis, we also need to make a seperate field so that we can check if the collision cause any injuries or fatalities.

In [29]:
vehicle_data <- vehicle_data %>%
  mutate(INJURY_OR_FATALITY = ifelse(PERSON_INJURY %in% c("Injured", "Killed"), TRUE, FALSE))

After a quick check on the count of TYPE_OF_VEHICLES with our newly created INJURY_OR_FATALITY column

In [ ]:
table(vehicle_data$TYPE_OF_VEHICLE, vehicle_data$INJURY_OR_FATALITY)

We can see that ***Livery*** (Limo services) has less than 30 instances, meaning it might cause our model some errors as a variable. We will combine ***Livery*** with Taxi services for this analysis as they both functionally do similar things, drive a paying client/passenger from one place to another.

In [31]:
library(forcats)
vehicle_data <- vehicle_data %>%
  mutate(TYPE_OF_VEHICLE = fct_collapse(TYPE_OF_VEHICLE, "Taxi/Livery" = c("Livery", "Taxi")))

When checking `PERSON_SEX`, we can see that it has 3 factors

In [ ]:
unique(vehicle_data$PERSON_SEX)

#Checks how many instances of unknown sex there are
sum(vehicle_data$PERSON_SEX == "U", na.rm = TRUE)

For our regression model we will want to use Male and Female for `PERSON_SEX`. We will filter out the unknown sex for this analysis. 

In [33]:
vehicle_data <- vehicle_data %>% 
  filter(PERSON_SEX != "U")

#### **NOTE: The following will show the processes to get the final model. If you would like to skip to the final model, run the code for the following steps and go to "Final Regression Model"**

Now it is time to make the first model, we will use a logistical regression model. We chose logistical regression as we are trying to find whether or not (True or False), the variables are factors for injury or fatality. 

In [ ]:
mod1 <- glm(INJURY_OR_FATALITY ~ BOROUGH+TYPE_OF_VEHICLE+PERSON_SEX+DRIVER_LICENSE_STATUS+PERSON_AGE, data = vehicle_data, family = "binomial")
summary(mod1)

#Calculate multicollinearity using the VIF (Variance Inflation Factor) function
vif_mod1 <- car::vif(mod1)
print(vif_mod1)

As we can see from the first model, we have variables that are statistically significant and many others that aren't. We will now do a **Chi-Square** and **Hosmer-Lemeshow** tests to check if the model is better at predicting outcomes than the null model and aligns with the data.

In [ ]:
null_mod1 <- glm(INJURY_OR_FATALITY ~ 1, data = vehicle_data, family = "binomial")
anova(null_mod1, mod1, test = "Chisq")

#Testing now to see how well the model fits with the data
library(ResourceSelection)
hoslem.test(vehicle_data$INJURY_OR_FATALITY, fitted(mod1))

We can see from the result that the **Chi-Squared** test came out as signficant, meaning that our model was better than the null model at predicting injury and fatalities. However we see the **Hosmer-Lemeshow** test came back significant, meaning our model doesn't fit the data inputted. This means that our model needs some work. 

To work on the current model, let us combine some of the non-significant variables. We will combine larger vehicles (ex: Vans, Trucks, Buses, Emergency vehicles) into one category called `Large Vehicles`. After we will re-run the model as `mod2`.

In [ ]:
vehicle_data$TYPE_OF_VEHICLE <- fct_collapse(vehicle_data$TYPE_OF_VEHICLE,
                                             "Large Vehicles" = c("Truck", "Pickup Truck", "Van", "Emergency Vehicle"))

mod2 <- glm(INJURY_OR_FATALITY ~ BOROUGH + TYPE_OF_VEHICLE + PERSON_SEX + 
              DRIVER_LICENSE_STATUS + PERSON_AGE, 
            data = vehicle_data, family = "binomial")
summary(mod2)

hoslem.test(vehicle_data$INJURY_OR_FATALITY, fitted(mod2))


We can see that the model still is not matching the data. Our only numerical data is `PERSON_AGE`. Let us check the graph to see if there is anything we can gather.

In [ ]:
age_summary <- vehicle_data %>%
  group_by(PERSON_AGE) %>%  
  summarise(TOTAL_CASES = n(), INJURY_OR_FATALITY_COUNT = sum(INJURY_OR_FATALITY)) %>%
  mutate(PROBABILITY_PER_AGE = INJURY_OR_FATALITY_COUNT / TOTAL_CASES)  

ggplot(age_summary, aes(x = PERSON_AGE, y = PROBABILITY_PER_AGE)) +
  geom_point(color = "red", size = 2) +  
  labs(title = "Likelihood of Injury or Fatality by Age", x = "Age", y = "Likelihood of Injury or Fatality") + theme_minimal()


There seems to be some outliers at the ends of the age spectrum as seen by the graph, but there is also a clear curve and parabolic relationship that `PERSON_AGE` has with injury and fatalities. We will adjust our model accordingly, adding the poly() function as well as filtering out any non-significant relationships between predictors.

# **Final Model**

In [ ]:
vehicle_data <- vehicle_data %>% 
  filter(TYPE_OF_VEHICLE != "Large Vehicles")

mod3 <- glm(INJURY_OR_FATALITY ~ BOROUGH + TYPE_OF_VEHICLE + PERSON_SEX + 
              DRIVER_LICENSE_STATUS + poly(PERSON_AGE,2), 
            data = vehicle_data, family = "binomial")

summary(mod3)

hoslem.test(vehicle_data$INJURY_OR_FATALITY, fitted(mod3))

null_mod2 <- glm(INJURY_OR_FATALITY ~ 1, data = vehicle_data, family = "binomial")

anova(null_mod2, mod3, test = "Chisq")

Success! We now have a model that aligns with the data and is signficant. 

## **Conclusions From Significant Predictors**:

Conclusions from Borough
* Manhattan has the lowest probability of injury/fatality (Estimate: -1.035, p < 0.001).
* The Bronx, Brooklyn, and Queens also show lower odds, though less significantly than Manhattan.
* Accidents in Manhattan are significantly less likely to result in injuries or fatalities compared to the baseline borough (Staten Island) which might be because of high traffic and less high speed collisions. Further research would be needed to fully explain what is causing this decrease in injuries or fatalities in the boroughs.

Conclusions from Vehicle Type
* Motorcycles have the highest risk (Estimate: 1.95, p < 0.001), meaning motorcycle crashes are much more likely to result in injury or death. Motorcycles don't have the same protections that other motor vehicles do, which could explain why this is the case. 
* Taxis/Livery vehicles also show a higher risk (Estimate: 0.28, p = 0.01).
* SUVs and Sedans have increased risk compared to the baseline vehicle type.
* Other vehicles show a slight negative effect (Estimate: -0.25, p = 0.047), suggesting they are less risky.

Conclusions from Sex of Driver
* Males are significantly less likely to be injured or killed in an accident (Estimate: -0.34, p < 0.001). This might warrant further research as to why this might be the case. 
* Females appear to have a higher risk of injury or fatality in accidents.

Conclusion from Driver’s License Status
* Licensed drivers are significantly less likely to be injured (Estimate: -0.41, p < 0.001).
* Permit drivers shows no strong effect (p = 0.237).

Conclusion from Age
* Age has a significant non linear effect, modeled as a parabolic function in the regression model
* The first order polynomial has a strong negative impact (Estimate: -32.87, p < 0.001).
* The second order term has a positive effect (Estimate: 11.85, p = 0.0003).
* This suggests that injury risk is higher at younger and older ages, following a U-shaped relationship. This makes sense intuitively as younger drivers tend to be risker with their driving and older drivers could be more prone to injury. 

# **Data Visualizations**

Graphing the relationship between borough and injury/fatalities per 100,000 residents shows us the conclusion we got from our regression model. 

In [ ]:

borough_pop <- data.frame(
  BOROUGH = c("BRONX", "BROOKLYN", "MANHATTAN", "QUEENS", "STATEN ISLAND"),
  POPULATION = c(1379946, 2590516, 1596273, 2278029, 491133)
)

borough_pop$BOROUGH <- as.factor(borough_pop$BOROUGH)
vehicle_data <- merge(vehicle_data, borough_pop, by = "BOROUGH")

injury_or_fatality_summary <- vehicle_data %>%
  group_by(BOROUGH) %>% summarize(INJURY_OR_FATALITY_COUNT = sum(INJURY_OR_FATALITY),
                                  POPULATION = first(POPULATION)) %>%
  mutate(RATE_PER_100K = (INJURY_OR_FATALITY_COUNT / POPULATION) * 100000)

ggplot(injury_or_fatality_summary, aes(x = BOROUGH, y = RATE_PER_100K, fill = BOROUGH)) +
  geom_bar(stat = "identity") + labs(title = "Injuries or Fatalities per 100,000 Residents by Borough",
                                     x = "Borough", y = "Rate per 100,000 Residents") + theme_minimal() + theme(legend.position = "none")



Graphing age again shows us clearly how younger and older drivers have higher injury risk. 


In [ ]:
age_summary <- vehicle_data %>%
  group_by(PERSON_AGE) %>%  
  summarise(TOTAL_CASES = n(), INJURY_OR_FATALITY_COUNT = sum(INJURY_OR_FATALITY)) %>%
  mutate(PROBABILITY_PER_AGE = INJURY_OR_FATALITY_COUNT / TOTAL_CASES)  

ggplot(age_summary, aes(x = PERSON_AGE, y = PROBABILITY_PER_AGE)) +
  geom_point(color = "red", size = 2) +  
  labs(title = "Likelihood of Injury or Fatality by Age", x = "Age", y = "Likelihood of Injury or Fatality") + theme_minimal()


# **Final Thoughts**
Overall, this analysis identified key factors influencing injury and fatality risks in NYC. The borough of the accident, type of vehicle involved, driver's sex, and license status all play a significant role in determining the likelihood of an injury.

From the findings, Manhattan stands out as the safest borough in terms of injury rates, likely due to lower speed collisions in traffic. Motorcycles pose by far the highest risk, reinforcing the need for improved safety measures for motorcyclists. Age has a U shaped relationship with injury/fatality, showing increased risk of both younger and older drivers.

While many of these results align with intuition and observations, further research is needed to expand on the underlying reasons of some trends, such as why female drivers have higher injury rates than males or why unlicensed drivers appear less likely to be reported as injured. Future work could incorporate additional factors like time of day, brand of the vehicle, or traffic laws to refine risk assessments even further.